In [6]:
# used_car_price_pipeline.py
# Full pipeline for Used Car Price Prediction
# Designed to run in a notebook or as a script. Adjust CURRENT_YEAR if required.

import os, glob, re
import numpy as np, pandas as pd, matplotlib.pyplot as plt, joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# ---------- Config ----------
DATA_DIR = "/content/drive/MyDrive/Project: Used Car Price Prediction using Vehicle Dataset/Car details"
CURRENT_YEAR = 2025   # adjust if necessary
PLOT_DIR = os.path.join(DATA_DIR, "car_price_plots")
os.makedirs(PLOT_DIR, exist_ok=True)
# ----------------------------

def find_dataset(data_dir=DATA_DIR):
    csvs = glob.glob(os.path.join(data_dir, "*.csv"))
    # prefer files with 'car' or 'car details' in name
    prioritized = [p for p in csvs if re.search(r"car", os.path.basename(p), re.I)]
    return prioritized[0] if prioritized else (csvs[0] if csvs else None)

def standardize_column_names(df):
    # Lowercase and strip underscores/spaces
    df = df.copy()
    df.columns = [re.sub(r'[^0-9a-z_]', '_', re.sub(r'\s+', '_', c.strip().lower())
    ) for c in df.columns]
    # map common variants to standard names
    colmap = {}
    name_variants = ['name', 'car_name', 'carname', 'name_of_car']
    year_variants = ['year', 'manufacture_year', 'yr']
    price_variants = ['selling_price','sellingprice','price','sell_price','sellingprice_in_rupees','selling_price_(in_rupees)']
    km_variants = ['km_driven','kms_driven','km_driven_','km_driven_', 'present_km', 'present_kms_driven', 'present_price', 'kms']
    fuel_variants = ['fuel','fuel_type','fueltype']
    seller_variants = ['seller_type','seller_type_','seller']
    trans_variants = ['transmission','trans','transmission_type']
    owner_variants = ['owner','ownership','owner_type']

    def find_and_map(variants, target):
        for v in variants:
            matches = [c for c in df.columns if v == c or v in c]
            if matches:
                colmap[matches[0]] = target
                return True
        return False

    find_and_map(name_variants, 'name')
    find_and_map(year_variants, 'year')
    find_and_map(price_variants, 'selling_price')
    # for km variants try best match (search for 'km' or 'kmdriven' etc)
    km_candidates = [c for c in df.columns if re.search(r'km|kms|kmdriven|kmdr', c)]
    if km_candidates:
        colmap[km_candidates[0]] = 'km_driven'
    find_and_map(fuel_variants, 'fuel')
    find_and_map(seller_variants, 'seller_type')
    find_and_map(trans_variants, 'transmission')
    find_and_map(owner_variants, 'owner')

    df = df.rename(columns=colmap)
    return df

def clean_price_column(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float)): return float(x)
    s = str(x).lower().strip()
    s = s.replace(",", "").replace("rs", "").replace("inr", "").strip()
    # handle lakhs/crores
    if "lakh" in s or "lac" in s:
        num = re.findall(r"[\d\.]+", s)
        if num:
            return float(num[0]) * 100000
    if "crore" in s:
        num = re.findall(r"[\d\.]+", s)
        if num:
            return float(num[0]) * 10000000
    # otherwise extract numeric
    num = re.findall(r"[\d\.]+", s)
    if num:
        return float(num[0])
    return np.nan

def load_and_prepare(path):
    df = pd.read_csv(path, encoding='latin1')
    df = standardize_column_names(df)
    # Ensure expected columns exist
    expected = ['name','year','selling_price','km_driven','fuel','seller_type','transmission','owner']
    # Convert selling_price
    if 'selling_price' in df.columns:
        if df['selling_price'].dtype == object or not np.issubdtype(df['selling_price'].dtype, np.number):
            df['selling_price'] = df['selling_price'].apply(clean_price_column)
            # Heuristic: if median is small (<1000) try treat as lakhs -> convert to rupees
            if df['selling_price'].median() and df['selling_price'].median() < 1000:
                # only convert if original text didn't explicitly include 'lakh' or 'crore'
                if not df['selling_price'].astype(str).str.contains('lakh|lac|crore', case=False).any():
                    df['selling_price'] = df['selling_price'] * 100000
    # km_driven conversion
    if 'km_driven' in df.columns:
        if df['km_driven'].dtype == object:
            df['km_driven'] = df['km_driven'].astype(str).str.replace(",", "").str.extract(r"(\d+)").astype(float)
    # year numeric
    if 'year' in df.columns:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
    # drop duplicates and rows without target
    df = df.drop_duplicates()
    df = df.dropna(subset=['selling_price'])
    # feature: car_age
    if 'year' in df.columns:
        df['car_age'] = CURRENT_YEAR - df['year']
    else:
        # if year not available try to parse from name (rare)
        df['car_age'] = np.nan
    # basic filtering
    df = df[df['car_age'].ge(0) & df['car_age'].le(60)]
    if 'km_driven' in df.columns:
        df = df[df['km_driven'].ge(0)]
    # remove top 0.5% price outliers
    if 'selling_price' in df.columns:
        upper = df['selling_price'].quantile(0.995)
        df = df[df['selling_price'] <= upper]
    return df

def create_and_save_eda(df, plot_dir=PLOT_DIR):
    # price distribution
    plt.figure(figsize=(8,5))
    plt.hist(df['selling_price'], bins=50)
    plt.title("Selling Price Distribution")
    plt.xlabel("Selling Price")
    plt.ylabel("Count")
    plt.tight_layout()
    p1 = os.path.join(plot_dir, "price_distribution.png")
    plt.savefig(p1)
    plt.close()

    # price vs age
    if 'car_age' in df.columns:
        plt.figure(figsize=(8,5))
        plt.scatter(df['car_age'], df['selling_price'], s=6)
        plt.title("Selling Price vs Car Age")
        plt.xlabel("Car Age (years)")
        plt.ylabel("Selling Price")
        plt.tight_layout()
        p2 = os.path.join(plot_dir, "price_vs_age.png")
        plt.savefig(p2)
        plt.close()

    # price vs km_driven
    if 'km_driven' in df.columns:
        plt.figure(figsize=(8,5))
        plt.scatter(df['km_driven'], df['selling_price'], s=6)
        plt.title("Selling Price vs Km Driven")
        plt.xlabel("Km Driven")
        plt.ylabel("Selling Price")
        plt.tight_layout()
        p3 = os.path.join(plot_dir, "price_vs_km.png")
        plt.savefig(p3)
        plt.close()

    return [p for p in (p1, (p2 if 'car_age' in df.columns else None), (p3 if 'km_driven' in df.columns else None)) if p]

def prepare_features(df):
    # features: baseline numeric + categorical set
    numeric_features = []
    if 'km_driven' in df.columns:
        numeric_features.append('km_driven')
    if 'car_age' in df.columns:
        numeric_features.append('car_age')

    categorical_features = [c for c in ['fuel', 'seller_type', 'transmission', 'owner'] if c in df.columns]

    X = df[numeric_features + categorical_features].copy()
    y = df['selling_price'].astype(float).copy()

    # Fill categorical missing
    for c in categorical_features:
        X[c] = X[c].fillna('Unknown').astype(str)

    # Preprocessor
    num_transformer = Pipeline([('scaler', StandardScaler())])
    cat_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer([
        ('num', num_transformer, numeric_features),
        ('cat', cat_transformer, categorical_features)
    ])

    return X, y, preprocessor, numeric_features, categorical_features

def train_and_evaluate(X, y, preprocessor, numeric_features, categorical_features):
    # Models to compare
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingRegressor(random_state=42)
    }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = []
    fitted = {}
    for name, model in models.items():
        pipe = Pipeline([('preprocessor', preprocessor), ('model', model)])
        pipe.fit(X_train, y_train)
        preds = pipe.predict(X_test)
        r2 = r2_score(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        rmse = mean_squared_error(y_test, preds) # Removed squared=False
        results.append({'model': name, 'r2': r2, 'mae': mae, 'rmse': rmse})
        fitted[name] = pipe
        print(f"[{name}] R2={r2:.4f}, MAE={mae:.2f}, RMSE={rmse:.2f}")

    # Hyperparameter tuning example for RandomForest
    rf_pipe = Pipeline([('preprocessor', preprocessor), ('model', RandomForestRegressor(random_state=42, n_jobs=-1))])
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__max_depth': [5, 10, None],
        'model__min_samples_split': [2, 5]
    }
    gs = GridSearchCV(rf_pipe, param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
    gs.fit(X_train, y_train)
    best_rf = gs.best_estimator_
    rf_preds = best_rf.predict(X_test)
    rf_r2 = r2_score(y_test, rf_preds)
    rf_mae = mean_absolute_error(y_test, rf_preds)
    rf_rmse = mean_squared_error(y_test, rf_preds) # Removed squared=False
    results.append({'model': 'RandomForest_tuned', 'r2': rf_r2, 'mae': rf_mae, 'rmse': rf_rmse})
    fitted['RandomForest_tuned'] = best_rf
    print(f"[RandomForest_tuned] Best params: {gs.best_params_}")
    print(f"[RandomForest_tuned] R2={rf_r2:.4f}, MAE={rf_mae:.2f}, RMSE={rf_rmse:.2f}")

    results_df = pd.DataFrame(results).sort_values('rmse').reset_index(drop=True)
    return results_df, fitted, X_test, y_test

def plot_pred_vs_actual(model_pipe, X_test, y_test, fname):
    y_pred = model_pipe.predict(X_test)
    plt.figure(figsize=(8,6))
    plt.scatter(y_test, y_pred, s=6)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], linewidth=2)
    plt.xlabel("Actual Selling Price")
    plt.ylabel("Predicted Selling Price")
    plt.title("Predicted vs Actual")
    plt.tight_layout()
    plt.savefig(fname)
    plt.close()

def extract_feature_importance(model_pipe, numeric_features, categorical_features):
    # Only works for tree-based models with feature_importances_
    try:
        # Get onehot encoder feature names
        ohe = model_pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
        ohe_names = list(ohe.get_feature_names_out(categorical_features))
        feat_names = numeric_features + ohe_names
        importances = model_pipe.named_steps['model'].feature_importances_
        fi = pd.DataFrame({'feature': feat_names, 'importance': importances}).sort_values('importance', ascending=False)
        return fi
    except Exception:
        return None

# ---------------- Main flow ----------------
dataset = find_dataset()
if dataset is None:
    raise FileNotFoundError("No CSV dataset found in /mnt/data. Upload your CSV and retry.")

print("Using dataset:", dataset)
df = load_and_prepare(dataset)
print("Rows after cleaning:", len(df))
print("Columns available:", df.columns.tolist())

# quick peek
print(df.head(5).to_string())

# EDA plots (saved)
plots = create_and_save_eda(df)
print("Saved EDA plots:", plots)

# features & preprocessing
X, y, preprocessor, numeric_features, categorical_features = prepare_features(df)

# ensure we have at least one numeric or categorical feature
if X.shape[1] == 0:
    raise ValueError("No usable features found after preparing dataset. Check dataset columns.")

results_df, fitted_models, X_test, y_test = train_and_evaluate(X, y, preprocessor, numeric_features, categorical_features)

print("Model comparison (sorted by RMSE):")
print(results_df.to_string(index=False))

# Choose best model
best_row = results_df.iloc[0]
best_name = best_row['model']
best_model = fitted_models[best_name]

# Save predicted vs actual
pred_plot_path = os.path.join(PLOT_DIR, f"pred_vs_actual_{best_name}.png")
plot_pred_vs_actual(best_model, X_test, y_test, pred_plot_path)
print("Saved Pred vs Actual plot to", pred_plot_path)

# Feature importance if available
fi = extract_feature_importance(best_model, numeric_features, categorical_features)
if fi is not None:
    fi_path = os.path.join(DATA_DIR, "feature_importance.csv")
    fi.to_csv(fi_path, index=False)
    print("Feature importance saved to", fi_path)
    print(fi.head(10).to_string(index=False))
else:
    print("Feature importance not available for the selected best model (likely not tree-based).")

# Save best model
model_path = os.path.join(DATA_DIR, f"best_model_{best_name}.joblib")
joblib.dump(best_model, model_path)
print("Saved best model to", model_path)

# Save summary
summary = {
    "dataset_file": os.path.basename(dataset),
    "rows_after_cleaning": int(len(df)),
    "features_used": numeric_features + categorical_features,
    "best_model": best_name,
    "best_model_metrics": best_row.to_dict(),
    "plots_dir": PLOT_DIR,
    "saved_model": model_path
}
summary_txt = os.path.join(DATA_DIR, "car_price_pipeline_summary.txt")
with open(summary_txt, "w") as f:
    for k,v in summary.items():
        f.write(f"{k}: {v}\n")
print("Saved summary to", summary_txt)

Using dataset: /content/drive/MyDrive/Project: Used Car Price Prediction using Vehicle Dataset/Car details/CAR DETAILS FROM CAR DEKHO.csv
Rows after cleaning: 3559
Columns available: ['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'car_age']
                       name  year  selling_price  km_driven    fuel seller_type transmission         owner  car_age
0             Maruti 800 AC  2007          60000      70000  Petrol  Individual       Manual   First Owner       18
1  Maruti Wagon R LXI Minor  2007         135000      50000  Petrol  Individual       Manual   First Owner       18
2      Hyundai Verna 1.6 SX  2012         600000     100000  Diesel  Individual       Manual   First Owner       13
3    Datsun RediGO T Option  2017         250000      46000  Petrol  Individual       Manual   First Owner        8
4     Honda Amaze VX i-DTEC  2014         450000     141000  Diesel  Individual       Manual  Second Owner       11
Saved EDA plots